In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter

warnings.filterwarnings("ignore")


In [ ]:
DATA_PATH = "/kaggle/input/ventilator-pressure-prediction/"

sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')
df_train = pd.read_csv(DATA_PATH + 'train.csv')
df_test = pd.read_csv(DATA_PATH + 'test.csv')


#df = df_train[df_train['breath_id'] < 5].reset_index(drop=True)

In [ ]:
df_train

In [ ]:
df_train['u_in_cumsum'] = (df_train['u_in']).groupby(df_train['breath_id']).cumsum()

#df_test['u_in_cumsum'] = (df_test['u_in']).groupby(df_test['breath_id']).cumsum()
df_train['u_in_lag'] = df_train['u_in'].shift(2)
df_train = df_train.fillna(0)

#df_test['u_in_lag'] = df_test['u_in'].shift(2)
#df_test = df_test.fillna(0)

In [ ]:
#reseting index
df_train['time_step']=pd.to_datetime(df_train['time_step'])
df_train.set_index('time_step', inplace=True)
df_train.sort_values(by=['time_step','breath_id'])
df_train.info()

In [ ]:
limit=int(df_train.shape[0]*0.9)

In [ ]:
X=df_train.drop(["pressure",'id'],axis=1)
y=df_train['pressure']

In [ ]:
X.shape

In [ ]:
X_train=X[:limit].to_numpy()
X_valid=X[limit:].to_numpy()
y_train=y[:limit].to_numpy()
y_valid=y[limit:].to_numpy()

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid = X_valid.reshape((X_valid.shape[0], 1, X_valid.shape[1]))

In [ ]:
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
model = Sequential()
model.add(LSTM(100,return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])) )
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(40))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',patience=10, verbose=1,mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, verbose=1,
    mode='min')

In [ ]:
# fit network
history = model.fit(X_train, y_train, epochs=500, batch_size=1024, 
                    validation_data=(X_valid, y_valid), verbose=1, 
                    callbacks=[es,plateau],shuffle=False)

In [ ]:
df_test = pd.read_csv(DATA_PATH + 'test.csv')
df_test['u_in_cumsum'] = (df_test['u_in']).groupby(df_test['breath_id']).cumsum()
df_test['u_in_lag'] = df_test['u_in'].shift(2)
df_test = df_test.fillna(0)
df_test.sort_values(by=['time_step','breath_id'])

In [ ]:
#reseting index
df_test['time_step']=pd.to_datetime(df_test['time_step'])
df_test.set_index('time_step', inplace=True)
df_test.info()

In [ ]:
test=df_test.drop('id',axis=1)

In [ ]:
test=df_test.drop(['id'],axis=1)
test=test.to_numpy()
test = test.reshape((test.shape[0], 1, test.shape[1]))

In [ ]:
preds=model.predict(test)
df_test['pressure']=preds

In [ ]:
df_test=df_test.sort_values(by=['id'])

In [ ]:
output=df_test[['id','pressure']]

In [ ]:
output.to_csv('submission.csv',index=False)
output

In [ ]:
output.describe()